In [18]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.layers import Flatten,Dense
import glob
import os
from PIL import Image
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
img_size=(224,224,3)

In [3]:
vgg=VGG19(include_top=False,input_shape=img_size)

In [4]:
for i in vgg.layers:
    i.trainable=False

In [5]:
out=glob.glob('dataset/train/*')

In [6]:
flat=Flatten()(vgg.output)

In [7]:
layer=Dense(2,activation='softmax')(Dense())(flat)

In [8]:
model=Model(inputs=vgg.input,outputs=layer)

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
Img_train_data=ImageDataGenerator(rescale=1/255,zoom_range=0.2,shear_range=0.2,horizontal_flip=True)
Img_test_data=ImageDataGenerator(rescale=1/255)

In [12]:
train_data=Img_train_data.flow_from_directory('dataset/Train/',target_size=(224,224),batch_size=32)

Found 416 images belonging to 2 classes.


In [13]:
test_data=Img_test_data.flow_from_directory('dataset/Test/',target_size=(224,224))

Found 134 images belonging to 2 classes.


In [14]:
model.fit_generator(train_data,steps_per_epoch=len(train_data),epochs=10,validation_data=test_data,validation_steps=len(test_data))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
13/13 [==============================] - 217s 17s/step - loss: 0.9038 - accuracy: 0.5889 - val_loss: 0.5856 - val_accuracy: 0.7015
Epoch 2/10
13/13 [==============================] - 200s 15s/step - loss: 0.5247 - accuracy: 0.7308 - val_loss: 0.4916 - val_accuracy: 0.7537
Epoch 3/10
13/13 [==============================] - 196s 15s/step - loss: 0.3770 - accuracy: 0.8365 - val_loss: 0.6315 - val_accuracy: 0.6791
Epoch 4/10
13/13 [==============================] - 199s 15s/step - loss: 0.3959 - accuracy: 0.8389 - val_loss: 0.6673 - val_accuracy: 0.6493
Epoch 5/10
13/13 [==============================] - 206s 16s/step - loss: 0.2960 - accuracy: 0.8798 - val_loss: 0.4053 - val_accuracy: 0.7985
Epoch 6/10
13/13 [==============================] - 199s 15s/step - loss: 0.2684 - accuracy: 0.8918 - val_loss: 0.4022 - val_accuracy: 0.8209
Epoch 7/10
13/13 [==============================] - 197s 15s/step - loss

In [15]:
len(train_data)

13

In [17]:
k=model.to_json()
with open ('Malaria.json','w') as l:
    l.write(k)
model.save_weights('Malaria.h5')